In [2]:
import pandas as pd 
import requests 
import os
import requests
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder
#Helper function to make directory
def make_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

C:\Users\homan\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def get_SP500_symbols():
    # Ref: https://stackoverflow.com/a/75845569/
    url = 'https://en.m.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tickers = pd.read_html(url, attrs={'id': 'constituents'})[0]
    file = 'data/SP500.csv'
    tickers.to_csv(file, index=False)
    print('SP500 Saved')
    return tickers
df = get_SP500_symbols()

SP500 Saved


In [5]:
# Features to preprocess: Marketcap, Country, Ipoyear, Sector, Industry , Volume
# Onehot Encoding: Country, Industry, Sector
# Ordinal Encoding: Ipoyear, Marketcap, Volume
oe_target = ['GICS Sector', 'GICS Sub-Industry', 'Headquarters Location']
oe = OneHotEncoder(sparse_output=False)
oe.fit(df[oe_target])

#Save onehotencoder for later usage
with open('data/onehot_encoder_sp500.pkl', 'wb') as to_write:
    pickle.dump(oe, to_write)

transformed_data = oe.transform(df[oe_target])
transformed_df = pd.DataFrame(transformed_data, columns=oe.get_feature_names_out(oe_target))

df = df.drop(columns=oe_target)
df = pd.concat([df, transformed_df], axis=1)

In [7]:
display(df)

,Symbol,Security,Date added,CIK,Founded,GICS Sector_Communication Services,GICS Sector_Consumer Discretionary,GICS Sector_Consumer Staples,GICS Sector_Energy,GICS Sector_Financials,...,"Headquarters Location_Westlake, Ohio","Headquarters Location_Westlake, Texas","Headquarters Location_Westminster, Colorado","Headquarters Location_White Plains, New York","Headquarters Location_Wilmington, Delaware","Headquarters Location_Wilmington, Massachusetts","Headquarters Location_Winona, Minnesota","Headquarters Location_Woonsocket, Rhode Island","Headquarters Location_Worsley, United Kingdom","Headquarters Location_Zurich, Switzerland"
0,MMM,3M,1957-03-04,66740,1902,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AOS,A. O. Smith,2017-07-26,91142,1916,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ABT,Abbott,1957-03-04,1800,1888,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ABBV,AbbVie,2012-12-31,1551152,2013 (1888),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ACN,Accenture,2011-07-06,1467373,1989,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,2011-11-01,1524472,2011,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
499,YUM,Yum! Brands,1997-10-06,1041061,1997,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500,ZBRA,Zebra Technologies,2019-12-23,877212,1969,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
501,ZBH,Zimmer Biomet,2001-08-07,1136869,1927,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Ordinal: arrange by percentile cut off (4 Groups)
# Ipoyear, Marketcap 
ordinal_target = ['CIK']

df['CIK'] = pd.to_numeric(df['CIK'])

for target in ordinal_target:
    print(target)
    df[f'{target}_label'] = pd.qcut(df[target], 4, labels=False)

CIK


In [10]:
df.to_csv('data/SP500_preprocess.csv',index=False)

In [11]:
display(df)

,Symbol,Security,Date added,CIK,Founded,GICS Sector_Communication Services,GICS Sector_Consumer Discretionary,GICS Sector_Consumer Staples,GICS Sector_Energy,GICS Sector_Financials,...,"Headquarters Location_Westlake, Texas","Headquarters Location_Westminster, Colorado","Headquarters Location_White Plains, New York","Headquarters Location_Wilmington, Delaware","Headquarters Location_Wilmington, Massachusetts","Headquarters Location_Winona, Minnesota","Headquarters Location_Woonsocket, Rhode Island","Headquarters Location_Worsley, United Kingdom","Headquarters Location_Zurich, Switzerland",CIK_label
0,MMM,3M,1957-03-04,66740,1902,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,AOS,A. O. Smith,2017-07-26,91142,1916,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,ABT,Abbott,1957-03-04,1800,1888,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,ABBV,AbbVie,2012-12-31,1551152,2013 (1888),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,ACN,Accenture,2011-07-06,1467373,1989,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,2011-11-01,1524472,2011,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3
499,YUM,Yum! Brands,1997-10-06,1041061,1997,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
500,ZBRA,Zebra Technologies,2019-12-23,877212,1969,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
501,ZBH,Zimmer Biomet,2001-08-07,1136869,1927,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
